Questions:
1) Timeline is from pre-pandemic to 2023. Workers in which type of work was most insulated from rising inflation?
2) Residents of which area was most insulated from rising inflation (viz: maps). Data not available.
3) What category of items faced the highest inflation?
4) Was there a difference with unionized workers?
5) Look at rate of change YoY, MoM, and QoQ in each year (for CPI and EPI)

In [1]:
#import dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np



# Read in and Clean Employee Cost Index data downloaded from BLS

In [2]:
#Read in ECI dataset and create dataframe
eci_data = pd.read_excel('Resources/CPI_ECI_Datasets/eci_continuous_dataset.xlsx', sheet_name = 'Data')
eci_df = pd.DataFrame(eci_data)
eci_df.head()

,Ownership,Industry,Occupation,Estimate Type,Periodicity,Year,Period,Estimate,Estimate Footnote,Discontinued Series ID,Current Series ID,Ownership Code,SIC Code,NAICS Code,OCS Code,SOC Code,Estimate Code,Periodicity Code
0,Civilian workers,All industries,All occupations,Total compensation,"12-month percent change, current dollars",1982,June,7.5,NaN,ECU10001A,CIU1010000000000A,1,0,000000,0,000000,1,A
1,Civilian workers,All industries,All occupations,Total compensation,"12-month percent change, current dollars",1982,September,7.3,NaN,ECU10001A,CIU1010000000000A,1,0,000000,0,000000,1,A
2,Civilian workers,All industries,All occupations,Total compensation,"12-month percent change, current dollars",1982,December,6.4,NaN,ECU10001A,CIU1010000000000A,1,0,000000,0,000000,1,A
3,Civilian workers,All industries,All occupations,Total compensation,"12-month percent change, current dollars",1983,March,6.6,NaN,ECU10001A,CIU1010000000000A,1,0,000000,0,000000,1,A
4,Civilian workers,All industries,All occupations,Total compensation,"12-month percent change, current dollars",1983,June,6.5,NaN,ECU10001A,CIU1010000000000A,1,0,000000,0,000000,1,A


In [3]:
#view data types
eci_df.dtypes

Ownership                  object
Industry                   object
Occupation                 object
Estimate Type              object
Periodicity                object
Year                        int64
Period                     object
Estimate                  float64
Estimate Footnote         float64
Discontinued Series ID     object
Current Series ID          object
Ownership Code              int64
SIC Code                    int64
NAICS Code                 object
OCS Code                    int64
SOC Code                   object
Estimate Code               int64
Periodicity Code           object
dtype: object

In [4]:
#make period integer to combine with year later
#create dictionary 
month_integer = {'January':1, 'February':2, 'March':3, 'April':4, 'May':5, 'June':6, 'July':7, 'August':8, 'September':9, 'October':10, 'November':11,
                 'December':12}
#add column where period is an integer
eci_df['Month'] = [month_integer[month] for month in eci_df['Period']]

eci_df.head()

,Ownership,Industry,Occupation,Estimate Type,Periodicity,Year,Period,Estimate,Estimate Footnote,Discontinued Series ID,Current Series ID,Ownership Code,SIC Code,NAICS Code,OCS Code,SOC Code,Estimate Code,Periodicity Code,Month
0,Civilian workers,All industries,All occupations,Total compensation,"12-month percent change, current dollars",1982,June,7.5,NaN,ECU10001A,CIU1010000000000A,1,0,000000,0,000000,1,A,6
1,Civilian workers,All industries,All occupations,Total compensation,"12-month percent change, current dollars",1982,September,7.3,NaN,ECU10001A,CIU1010000000000A,1,0,000000,0,000000,1,A,9
2,Civilian workers,All industries,All occupations,Total compensation,"12-month percent change, current dollars",1982,December,6.4,NaN,ECU10001A,CIU1010000000000A,1,0,000000,0,000000,1,A,12
3,Civilian workers,All industries,All occupations,Total compensation,"12-month percent change, current dollars",1983,March,6.6,NaN,ECU10001A,CIU1010000000000A,1,0,000000,0,000000,1,A,3
4,Civilian workers,All industries,All occupations,Total compensation,"12-month percent change, current dollars",1983,June,6.5,NaN,ECU10001A,CIU1010000000000A,1,0,000000,0,000000,1,A,6


In [5]:
#view columns
eci_df.columns

Index(['Ownership ', 'Industry', 'Occupation', 'Estimate Type', 'Periodicity',
       'Year', 'Period', 'Estimate', 'Estimate Footnote',
       'Discontinued Series ID', 'Current Series ID', 'Ownership Code',
       'SIC Code', 'NAICS Code', 'OCS Code', 'SOC Code', 'Estimate Code',
       'Periodicity Code', 'Month'],
      dtype='object')

In [6]:
#clean up columns (e.g., Ownership has a space at the end)
eci_df.columns = eci_df.columns.str.strip()
eci_df.columns

Index(['Ownership', 'Industry', 'Occupation', 'Estimate Type', 'Periodicity',
       'Year', 'Period', 'Estimate', 'Estimate Footnote',
       'Discontinued Series ID', 'Current Series ID', 'Ownership Code',
       'SIC Code', 'NAICS Code', 'OCS Code', 'SOC Code', 'Estimate Code',
       'Periodicity Code', 'Month'],
      dtype='object')

In [7]:
#create period + year column
eci_df['Period Year'] = pd.to_datetime(eci_df['Year'].astype(str) + eci_df['Month'].astype(str), format = '%Y%m')
#drop columns with codes where there are corresponding labels available
eci_df = eci_df.drop(columns = ['Ownership Code', 'SIC Code', 'NAICS Code', 'OCS Code', 'SOC Code', 'Estimate Code', 'Periodicity Code', 'Year', 'Period', 'Month'])
#reorder columns
eci_df = eci_df[['Period Year', 'Industry', 'Ownership', 'Occupation', 'Estimate Type', 'Periodicity', 'Estimate', 'Estimate Footnote', 'Discontinued Series ID',
        'Current Series ID']]
eci_df.head()

,Period Year,Industry,Ownership,Occupation,Estimate Type,Periodicity,Estimate,Estimate Footnote,Discontinued Series ID,Current Series ID
0,1982-06-01,All industries,Civilian workers,All occupations,Total compensation,"12-month percent change, current dollars",7.5,NaN,ECU10001A,CIU1010000000000A
1,1982-09-01,All industries,Civilian workers,All occupations,Total compensation,"12-month percent change, current dollars",7.3,NaN,ECU10001A,CIU1010000000000A
2,1982-12-01,All industries,Civilian workers,All occupations,Total compensation,"12-month percent change, current dollars",6.4,NaN,ECU10001A,CIU1010000000000A
3,1983-03-01,All industries,Civilian workers,All occupations,Total compensation,"12-month percent change, current dollars",6.6,NaN,ECU10001A,CIU1010000000000A
4,1983-06-01,All industries,Civilian workers,All occupations,Total compensation,"12-month percent change, current dollars",6.5,NaN,ECU10001A,CIU1010000000000A


In [8]:
#read ECI file to csv
eci_df.to_csv('Output/csv_files/eci.csv')

# Read in and Clean Consumer Price Index data downloaded from BLS

## This section reads in CPI data by item

In [9]:
#Read in CPI dataset and create dataframe. Reuse code for each of 2013-2023 excel files + save to csv at end
cpi_data = pd.read_excel('Resources/CPI_ECI_Datasets/cpi-u_2013.xlsx', sheet_name = 'US', header = 4)
cpi_df = pd.DataFrame(cpi_data)
cpi_df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Dec.\n2012,Jan.\n2013,Feb.\n2013,Mar.\n2013,Apr.\n2013,May\n2013,Jun.\n2013,...,Oct.\n2013-\nNov.\n2013.1,Nov.\n2013-\nDec.\n2013.1,Seasonally adjusted effect on All Items\nNov. 2013-\nDec. 2013(1),"Standard error, median price change(2)",Largest (L) or Smallest (S) seasonally adjusted change since:(3),Unnamed: 39,Unadjusted effect on All Items\nDec. 2012-\nDec. 2013(1),"Standard error, median price change(2).1",Largest (L) or Smallest (S) unadjusted change since:(3),Unnamed: 43
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Date,Percent change,NaN,NaN,Date,Percent change
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,All items,100.000,229.601,230.280,232.166,232.773,232.531,232.945,233.504,...,0.0,0.3,NaN,0.04,L-Jun. 2013,0.5,NaN,0.08,L-Aug. 2013,1.5
3,1.0,Food,14.233,235.390,236.341,236.301,236.332,236.841,236.526,236.792,...,0.1,0.1,0.007,0.08,–,–,0.151,0.14,S-Aug. 2010,1
4,2.0,Food at home,8.497,232.901,234.240,234.033,233.777,234.082,233.302,233.499,...,0.0,0.0,0.000,0.12,–,–,0.033,0.18,S-Jun. 2010,0.2


In [10]:
#view column names
cpi_df.columns

Index(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Dec.\n2012', 'Jan.\n2013',
       'Feb.\n2013', 'Mar.\n2013', 'Apr.\n2013', 'May\n2013', 'Jun.\n2013',
       'Jul.\n2013', 'Aug.\n2013', 'Sep.\n2013', 'Oct.\n2013', 'Nov.\n2013',
       'Dec.\n2013', 'Dec.\n2012.1', 'Jan.\n2013.1', 'Feb.\n2013.1',
       'Mar.\n2013.1', 'Apr.\n2013.1', 'May\n2013.1', 'Jun.\n2013.1',
       'Jul.\n2013.1', 'Aug.\n2013.1', 'Sep.\n2013.1', 'Oct.\n2013.1',
       'Nov.\n2013.1', 'Dec.\n2013.1', 'Dec.\n2012-\nDec.\n2013',
       'Sep.\n2013-\nOct.\n2013', 'Oct.\n2013-\nNov.\n2013',
       'Nov.\n2013-\nDec.\n2013', 'Sep.\n2013-\nOct.\n2013.1',
       'Oct.\n2013-\nNov.\n2013.1', 'Nov.\n2013-\nDec.\n2013.1',
       'Seasonally adjusted effect on All Items\nNov. 2013-\nDec. 2013(1)',
       'Standard error, median price change(2)',
       'Largest (L) or Smallest (S) seasonally adjusted change since:(3)',
       'Unnamed: 39',
       'Unadjusted effect on All Items\nDec. 2012-\nDec. 2013(1)',
       'Standard

In [11]:
#view length of columns to determine end column to drop
len(cpi_df.columns)

44

In [12]:
#drop columns that will not be used where Unnamed 0 = indent level, Unnamed 2 = relative importance, Dec.\Year-1 = previous year
#will be using unadjusted for both CPI and ECI so deleting seasonally adjusted data (month\nYear.1) and all columns after. Starts at column indexed 16
drop_columns = ['Unnamed: 0', 'Unnamed: 2', 'Dec.\n2012'] + list(cpi_df.columns[16:44])
cpi_df = cpi_df.drop(drop_columns, axis = 1)
cpi_df

,Unnamed: 1,Jan.\n2013,Feb.\n2013,Mar.\n2013,Apr.\n2013,May\n2013,Jun.\n2013,Jul.\n2013,Aug.\n2013,Sep.\n2013,Oct.\n2013,Nov.\n2013,Dec.\n2013
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,All items,230.280,232.166,232.773,232.531,232.945,233.504,233.596,233.877,234.149,233.546,233.069,233.049
3,Food,236.341,236.301,236.332,236.841,236.526,236.792,237.001,237.406,237.522,237.871,237.641,237.869
4,Food at home,234.240,234.033,233.777,234.082,233.302,233.499,233.591,233.999,234.045,234.418,233.639,233.802
...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,(15) Indexes on a December 2001=100 base.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
416,(16) Indexes on a December 1982=100 base.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
417,(17) Indexes on a December 1996=100 base.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
418,(18) Indexes on a December 1988=100 base.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
#create dictionary to rename column headers
headers = {'Unnamed: 1':'Expenditure category', 'Jan.\n2013':'2013-01-01', 'Feb.\n2013':'2013-02-01', 'Mar.\n2013':'2013-03-01', 'Apr.\n2013':'2013-04-01', 
           'May\n2013':'2013-05-01', 'Jun.\n2013':'2013-06-01', 'Jul.\n2013':'2013-07-01', 'Aug.\n2013':'2013-08-01', 'Sep.\n2013':'2013-09-01', 
           'Oct.\n2013':'2013-10-01', 'Nov.\n2013':'2013-11-01', 'Dec.\n2013':'2013-12-01'}
cpi_df = cpi_df.rename(columns = headers)
cpi_df

,Expenditure category,2013-01-01,2013-02-01,2013-03-01,2013-04-01,2013-05-01,2013-06-01,2013-07-01,2013-08-01,2013-09-01,2013-10-01,2013-11-01,2013-12-01
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,All items,230.280,232.166,232.773,232.531,232.945,233.504,233.596,233.877,234.149,233.546,233.069,233.049
3,Food,236.341,236.301,236.332,236.841,236.526,236.792,237.001,237.406,237.522,237.871,237.641,237.869
4,Food at home,234.240,234.033,233.777,234.082,233.302,233.499,233.591,233.999,234.045,234.418,233.639,233.802
...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,(15) Indexes on a December 2001=100 base.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
416,(16) Indexes on a December 1982=100 base.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
417,(17) Indexes on a December 1996=100 base.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
418,(18) Indexes on a December 1988=100 base.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
#drop first two empty rows and reset index
cpi_df = cpi_df.drop(index=[0,1])
cpi_df.reset_index(drop = True, inplace = True)
cpi_df

,Expenditure category,2013-01-01,2013-02-01,2013-03-01,2013-04-01,2013-05-01,2013-06-01,2013-07-01,2013-08-01,2013-09-01,2013-10-01,2013-11-01,2013-12-01
0,All items,230.280,232.166,232.773,232.531,232.945,233.504,233.596,233.877,234.149,233.546,233.069,233.049
1,Food,236.341,236.301,236.332,236.841,236.526,236.792,237.001,237.406,237.522,237.871,237.641,237.869
2,Food at home,234.240,234.033,233.777,234.082,233.302,233.499,233.591,233.999,234.045,234.418,233.639,233.802
3,Cereals and bakery products,269.078,269.304,269.504,271.388,271.041,271.716,271.279,271.285,270.894,270.303,269.750,269.267
4,Cereals and cereal products,231.286,232.913,231.326,232.455,234.095,234.768,234.717,233.053,231.881,232.665,232.021,231.100
...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,(15) Indexes on a December 2001=100 base.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
414,(16) Indexes on a December 1982=100 base.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
415,(17) Indexes on a December 1996=100 base.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
416,(18) Indexes on a December 1988=100 base.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
#find footnote row to delete all footnotes
footnote_row = cpi_df.index[cpi_df['Expenditure category'] == 'Footnotes:'].tolist()[0]
print(footnote_row)

398


In [16]:
#search tail to double check index
cpi_df.tail(22)

,Expenditure category,2013-01-01,2013-02-01,2013-03-01,2013-04-01,2013-05-01,2013-06-01,2013-07-01,2013-08-01,2013-09-01,2013-10-01,2013-11-01,2013-12-01
396,Personal care,213.249,213.934,214.754,215.041,214.854,215.058,214.915,215.289,215.41,215.38,215.749,216.109
397,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
398,Footnotes:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
399,(1) The 'effect' of an item category is a meas...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
400,(2) A statistic's margin of error is often exp...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
401,(3) If the current seasonally adjusted 1-month...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
402,(4) Not seasonally adjusted.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
403,(5) Indexes on a December 1997=100 base.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
404,(6) Special indexes based on a substantially s...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
405,(7) Indexes on a December 2007=100 base.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
#remove rows that contain footnotes
cpi_df = cpi_df.drop(index=range(footnote_row,len(cpi_df)))
#check that rows were dropped successfully
cpi_df.tail(20)

,Expenditure category,2013-01-01,2013-02-01,2013-03-01,2013-04-01,2013-05-01,2013-06-01,2013-07-01,2013-08-01,2013-09-01,2013-10-01,2013-11-01,2013-12-01
378,"Information technology, hardware and services(18)",8.555,8.632,8.679,8.630,8.581,8.535,8.459,8.445,8.459,8.422,8.401,8.392
379,Recreation(5),114.816,115.350,115.386,115.359,115.570,115.407,115.384,115.336,115.203,115.202,115.325,114.855
380,Video and audio(5),98.993,99.824,100.251,100.054,100.100,99.881,99.769,99.652,99.566,99.468,99.542,99.010
381,"Pets, pet products and services(5)",163.153,163.835,164.305,164.911,165.123,164.446,164.564,164.598,164.511,165.071,165.399,164.992
382,Photography(5),76.258,77.341,78.216,77.880,77.830,77.598,77.075,77.471,77.312,77.307,76.727,76.067
383,Food and beverages,236.183,236.230,236.267,236.761,236.474,236.726,236.957,237.348,237.444,237.794,237.585,237.820
384,Domestically produced farm food,241.509,241.505,241.166,241.277,240.887,241.054,241.291,241.667,241.644,241.924,241.207,241.358
385,Other services,325.993,327.276,327.576,327.216,327.535,327.607,328.160,329.637,330.723,330.964,331.202,331.067
386,Apparel less footwear,118.291,119.947,122.104,122.544,121.408,119.444,117.613,119.285,123.425,124.961,123.321,120.472
387,Fuels and utilities,220.228,220.992,220.251,221.382,224.847,230.506,230.899,229.850,230.318,225.244,223.566,224.407


In [24]:
#Unpivot columns
cpi_df_final = cpi_df.melt(['Expenditure category'], var_name = 'Date', value_name = 'Unadjusted CPI Index')

In [25]:
#Preview new df
cpi_df_final.head(20)

,Expenditure category,Date,Unadjusted CPI Index
0,All items,2013-01-01,230.280
1,Food,2013-01-01,236.341
2,Food at home,2013-01-01,234.240
3,Cereals and bakery products,2013-01-01,269.078
4,Cereals and cereal products,2013-01-01,231.286
5,Flour and prepared flour mixes,2013-01-01,257.057
6,Breakfast cereal(4),2013-01-01,225.185
7,"Rice, pasta, cornmeal(4)",2013-01-01,238.531
8,Rice(4)(5)(6),2013-01-01,167.810
9,Bakery products,2013-01-01,290.268


## This section reads in CPI data by region

In [26]:
#Read in CPI dataset by region and create dataframe. Reuse code for each of 2013-2023 excel files + save to csv at end
cpi_regional = pd.read_excel('Resources/CPI_ECI_Datasets/cpi-u_2013.xlsx', sheet_name = 'Regional', header = 4)
cpi_regional_df = pd.DataFrame(cpi_regional)
cpi_regional_df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Nov.\n2012,Dec.\n2012,Jan.\n2013,Feb.\n2013,Mar.\n2013,Apr.\n2013,May\n2013,...,Sep.\n2013,Oct.\n2013,Nov.\n2013,Dec.\n2013,Dec.\n2012.1,Oct.\n2013.1,Nov.\n2013.1,Nov.\n2012.1,Sep.\n2013.1,Oct.\n2013.2
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,U.S. city average,M,230.221,229.601,230.28,232.166,232.773,232.531,232.945,...,234.149,233.546,233.069,233.049,1.5,-0.2,0.0,1.2,-0.5,-0.2
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.0,Region and area size(2),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
#view columns
cpi_regional_df.columns

Index(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Nov.\n2012', 'Dec.\n2012',
       'Jan.\n2013', 'Feb.\n2013', 'Mar.\n2013', 'Apr.\n2013', 'May\n2013',
       'Jun.\n2013', 'Jul.\n2013', 'Aug.\n2013', 'Sep.\n2013', 'Oct.\n2013',
       'Nov.\n2013', 'Dec.\n2013', 'Dec.\n2012.1', 'Oct.\n2013.1',
       'Nov.\n2013.1', 'Nov.\n2012.1', 'Sep.\n2013.1', 'Oct.\n2013.2'],
      dtype='object')

In [28]:
#view length of columns to determine end column to drop
len(cpi_regional_df.columns)

23

In [29]:
#drop columns that will not be used where Unnamed:0 = indent level, Unnamed:2 = Pricing Schedule, Nov.\nYear-1, Dec.\nYear-1, all columns after Dec.\n2012.1 
#for MoM % change starting at column indexed 17 (MoM % change will be calculated in PowerBI and compared to original files to validate)
drop_columns = ['Unnamed: 0', 'Unnamed: 2'] + list(cpi_regional_df.columns[3:5]) + list(cpi_regional_df.columns[17:23])
cpi_regional_df = cpi_regional_df.drop(drop_columns, axis=1)
cpi_regional_df.head()

,Unnamed: 1,Jan.\n2013,Feb.\n2013,Mar.\n2013,Apr.\n2013,May\n2013,Jun.\n2013,Jul.\n2013,Aug.\n2013,Sep.\n2013,Oct.\n2013,Nov.\n2013,Dec.\n2013
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,U.S. city average,230.28,232.166,232.773,232.531,232.945,233.504,233.596,233.877,234.149,233.546,233.069,233.049
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Region and area size(2),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
#create dictionary to rename column headers
headers = {'Unnamed: 1':'Area', 'Jan.\n2013':'2013-01-01', 'Feb.\n2013':'2013-02-01', 'Mar.\n2013':'2013-03-01', 'Apr.\n2013':'2013-04-01', 
           'May\n2013':'2013-05-01', 'Jun.\n2013':'2013-06-01', 'Jul.\n2013':'2013-07-01', 'Aug.\n2013':'2013-08-01', 'Sep.\n2013':'2013-09-01', 
           'Oct.\n2013':'2013-10-01', 'Nov.\n2013':'2013-11-01', 'Dec.\n2013':'2013-12-01'}
cpi_regional_df = cpi_regional_df.rename(columns = headers)
cpi_regional_df.head()

,Area,2013-01-01,2013-02-01,2013-03-01,2013-04-01,2013-05-01,2013-06-01,2013-07-01,2013-08-01,2013-09-01,2013-10-01,2013-11-01,2013-12-01
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,U.S. city average,230.28,232.166,232.773,232.531,232.945,233.504,233.596,233.877,234.149,233.546,233.069,233.049
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Region and area size(2),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
#drop null rows as they're empty spacing for Excel aesthetics only
cpi_regional_df.dropna(axis = 0, how = 'all', inplace = True)
cpi_regional_df.head()

,Area,2013-01-01,2013-02-01,2013-03-01,2013-04-01,2013-05-01,2013-06-01,2013-07-01,2013-08-01,2013-09-01,2013-10-01,2013-11-01,2013-12-01
1,U.S. city average,230.280,232.166,232.773,232.531,232.945,233.504,233.596,233.877,234.149,233.546,233.069,233.049
3,Region and area size(2),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Northeast urban,247.277,248.665,248.719,248.464,248.584,248.851,249.411,249.858,250.231,249.320,249.503,249.567
6,"Size A - More than 1,500,000",249.154,250.535,250.771,250.427,250.553,250.937,251.523,251.797,252.365,251.254,251.535,251.650
7,"Size B/C - 50,000 to 1,500,000(3)",147.337,148.195,147.909,147.909,147.971,147.926,148.223,148.802,148.683,148.483,148.420,148.367


In [45]:
#reset index after dropping null rows
cpi_regional_df.reset_index(drop = True, inplace = True)
cpi_regional_df.head()

,Area,2013-01-01,2013-02-01,2013-03-01,2013-04-01,2013-05-01,2013-06-01,2013-07-01,2013-08-01,2013-09-01,2013-10-01,2013-11-01,2013-12-01
0,U.S. city average,230.280,232.166,232.773,232.531,232.945,233.504,233.596,233.877,234.149,233.546,233.069,233.049
1,Region and area size(2),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Northeast urban,247.277,248.665,248.719,248.464,248.584,248.851,249.411,249.858,250.231,249.320,249.503,249.567
3,"Size A - More than 1,500,000",249.154,250.535,250.771,250.427,250.553,250.937,251.523,251.797,252.365,251.254,251.535,251.650
4,"Size B/C - 50,000 to 1,500,000(3)",147.337,148.195,147.909,147.909,147.971,147.926,148.223,148.802,148.683,148.483,148.420,148.367


In [46]:
#find footnote row to delete all footnotes
footnote = cpi_regional_df.index[cpi_regional_df['Area'] == 'Footnotes:'].tolist()[0]
print(footnote)

35


In [47]:
#search tail to double check index
cpi_regional_df.tail(10)

,Area,2013-01-01,2013-02-01,2013-03-01,2013-04-01,2013-05-01,2013-06-01,2013-07-01,2013-08-01,2013-09-01,2013-10-01,2013-11-01,2013-12-01
34,"Seattle-Tacoma-Bremerton, WA",NaN,239.898,NaN,240.823,NaN,242.82,NaN,242.767,NaN,242.787,NaN,241.055
35,Footnotes:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,"(1) Foods, fuels, and several other items are ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37,(2) Regions defined as the four Census regions.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38,(3) Indexes on a December 1996=100 base.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39,(4) Indexes on a December 1986=100 base.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40,"(5) In addition, the following metropolitan ar...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41,(6) Indexes on a November 1996=100 base.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42,NOTE: Local area indexes are byproducts of the...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43,"NOTE: Index applies to a month as a whole, not...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
#remove rows that contain footnotes. Regional groupings were different as were size classifications between 2013-2023. 
#for those that had consistent groupings, basing was the same in footnotes
cpi_regional_df = cpi_regional_df.drop(index=range(footnote,len(cpi_regional_df)))
cpi_regional_df.tail(10)

,Area,2013-01-01,2013-02-01,2013-03-01,2013-04-01,2013-05-01,2013-06-01,2013-07-01,2013-08-01,2013-09-01,2013-10-01,2013-11-01,2013-12-01
25,"Cleveland-Akron, OH",215.102,NaN,216.946,NaN,217.738,NaN,219.251,NaN,218.380,NaN,216.772,NaN
26,"Dallas-Fort Worth, TX",213.696,NaN,216.465,NaN,215.670,NaN,216.979,NaN,217.068,NaN,215.450,NaN
27,"Washington-Baltimore, DC-MD-VA-WV(6)",150.845,NaN,152.188,NaN,151.908,NaN,152.657,NaN,153.532,NaN,153.160,NaN
28,"Atlanta, GA",NaN,215.009,NaN,215.205,NaN,217.337,NaN,218.781,NaN,216.505,NaN,216.017
29,"Detroit-Ann Arbor-Flint, MI",NaN,218.893,NaN,218.904,NaN,221.607,NaN,220.000,NaN,219.685,NaN,218.217
30,"Houston-Galveston-Brazoria, TX",NaN,205.716,NaN,207.506,NaN,207.882,NaN,208.575,NaN,207.788,NaN,209.814
31,"Miami-Fort Lauderdale, FL",NaN,238.524,NaN,238.124,NaN,237.815,NaN,237.438,NaN,238.858,NaN,239.417
32,"Philadelphia-Wilmington-Atlantic City, PA-NJ-D...",NaN,240.137,NaN,240.345,NaN,240.990,NaN,242.128,NaN,241.141,NaN,241.383
33,"San Francisco-Oakland-San Jose, CA",NaN,242.677,NaN,244.675,NaN,245.935,NaN,246.072,NaN,246.617,NaN,245.711
34,"Seattle-Tacoma-Bremerton, WA",NaN,239.898,NaN,240.823,NaN,242.820,NaN,242.767,NaN,242.787,NaN,241.055


In [27]:
#Unpivot columns
cpi_regional_final = cpi_regional_df.melt(['Area'], var_name = 'Date', value_name = 'Unadjusted CPI Index')

In [28]:
#preview new df
cpi_regional_final.head()

,Area,Date,Unadjusted CPI Index
0,U.S. city average,2013-01-01,230.280
1,Region and area size(2),2013-01-01,NaN
2,Northeast urban,2013-01-01,247.277
3,"Size A - More than 1,500,000",2013-01-01,249.154
4,"Size B/C - 50,000 to 1,500,000(3)",2013-01-01,147.337


In [29]:
#write cpi and regional cpi to csv
cpi_df_final.to_csv('Output/csv_files/2013_cpi_byitem.csv')
cpi_regional_final.to_csv('Output/csv_files/2013_cpi_regional.csv')

# Define function to data clean for all years using 2013 code above

In [25]:
def cpi_items(year):
    #Read in CPI dataset and create dataframe
    cpi_data = pd.read_excel(f'Resources/CPI_ECI_Datasets/cpi-u_{year}.xlsx', sheet_name = 'US', header = 4)
    cpi_df = pd.DataFrame(cpi_data)

    #drop columns that will not be used 
    drop_columns = ['Unnamed: 0', 'Unnamed: 2', f'Dec.\n{year-1}'] + list(cpi_df.columns[16:len(cpi_df.columns)])
    cpi_df = cpi_df.drop(drop_columns, axis = 1)

    #create dictionary to rename column headers
    headers = {'Unnamed: 1':'Expenditure category', f'Jan.\n{year}':f'{year}-01-01', f'Feb.\n{year}':f'{year}-02-01', 
               f'Mar.\n{year}':f'{year}-03-01', f'Apr.\n{year}':f'{year}-04-01', f'May\n{year}':f'{year}-05-01', 
               f'Jun.\n{year}':f'{year}-06-01', f'Jul.\n{year}':f'{year}-07-01', f'Aug.\n{year}':f'{year}-08-01', 
               f'Sep.\n{year}':f'{year}-09-01', f'Oct.\n{year}':f'{year}-10-01', f'Nov.\n{year}':f'{year}-11-01', 
               f'Dec.\n{year}':f'{year}-12-01'}
    cpi_df = cpi_df.rename(columns = headers)

    #drop first two empty rows and reset index
    cpi_df = cpi_df.drop(index=[0,1])
    cpi_df.reset_index(drop = True, inplace = True)

    #find footnote row to delete all footnotes
    footnote_row = cpi_df.index[cpi_df['Expenditure category'] == 'Footnotes:'].tolist()[0]

    #remove rows that contain footnotes
    cpi_df = cpi_df.drop(index=range(footnote_row,len(cpi_df)))

    #Unpivot columns
    cpi_df_final = cpi_df.melt(['Expenditure category'], var_name = 'Date', value_name = 'Unadjusted CPI Index')

    #write to csv
    cpi_df_final.to_csv(f'Output/csv_files/{year}_cpi_byitem.csv')

In [49]:
def cpi_regional(year):
    #Read in CPI dataset by region and create dataframe
    cpi_regional = pd.read_excel(f'Resources/CPI_ECI_Datasets/cpi-u_{year}.xlsx', sheet_name = 'Regional', header = 4)
    cpi_regional_df = pd.DataFrame(cpi_regional)

    #drop columns that will not be used 
    drop_columns = ['Unnamed: 0', 'Unnamed: 2'] + list(cpi_regional_df.columns[3:5])\
    + list(cpi_regional_df.columns[17:len(cpi_regional_df.columns)])
    cpi_regional_df = cpi_regional_df.drop(drop_columns, axis=1)

    #create dictionary to rename column headers
    headers = {'Unnamed: 1':'Area', f'Jan.\n{year}':f'{year}-01-01', f'Feb.\n{year}':f'{year}-02-01', 
               f'Mar.\n{year}':'{year}-03-01', f'Apr.\n{year}':f'{year}-04-01', f'May\n{year}':f'{year}-05-01', 
               f'Jun.\n{year}':f'{year}-06-01', f'Jul.\n{year}':f'{year}-07-01', f'Aug.\n{year}':f'{year}-08-01', 
               f'Sep.\n{year}':f'{year}-09-01', f'Oct.\n{year}':f'{year}-10-01', f'Nov.\n{year}':f'{year}-11-01', 
               f'Dec.\n{year}':f'{year}-12-01'}
    cpi_regional_df = cpi_regional_df.rename(columns = headers)

    #drop null rows as they're empty spacing for Excel aesthetics only
    cpi_regional_df.dropna(axis = 0, how = 'all', inplace = True)

    #reset index after dropping null rows
    cpi_regional_df.reset_index(drop = True, inplace = True)

    #find footnote row to delete all footnotes
    footnote = cpi_regional_df.index[cpi_regional_df['Area'] == 'Footnotes:'].tolist()[0]

    #remove rows that contain footnotes
    cpi_regional_df = cpi_regional_df.drop(index=range(footnote,len(cpi_regional_df)))

    #Unpivot columns
    cpi_regional_final = cpi_regional_df.melt(['Area'], var_name = 'Date', value_name = 'Unadjusted CPI Index')

    #write to csv
    cpi_regional_final.to_csv(f'Output/csv_files/{year}_cpi_regional.csv')

# Use functions to clean 2014-2023 files

In [27]:
#2014-2023 cpi by items
cpi_items(2014)
cpi_items(2015)
cpi_items(2016)
cpi_items(2017)
cpi_items(2018)
cpi_items(2019)
cpi_items(2020)
cpi_items(2021)
cpi_items(2022)
cpi_items(2023)

In [51]:
#2014-2023 cpi by region
cpi_regional(2014)
cpi_regional(2015)
cpi_regional(2016)
cpi_regional(2017)
cpi_regional(2018)
cpi_regional(2019)
cpi_regional(2020)
cpi_regional(2021)
cpi_regional(2022)
cpi_regional(2023)